In [3]:
import os
import yaml
from pydub import AudioSegment

# Define target folder and instrument IDs to remove
DIR_PATH = "slakh2100_flac_redux/reduced"

In [6]:
import time

# Iterate through each track folder
for track in os.listdir(DIR_PATH):
    track_path = os.path.join(DIR_PATH, track)
    metadata_path = os.path.join(track_path, "metadata.yaml")
    stems_path = os.path.join(track_path, "stems")
    mix_path = os.path.join(track_path, "mix.flac")
    
    if not os.path.isdir(track_path) or not os.path.exists(metadata_path):
        continue  
    
    print(f"Processing track: {track}")  # Debugging print
    
    with open(metadata_path, "r") as f:
        metadata = yaml.safe_load(f)
    
    if "stems" not in metadata:
        continue  
    
    stems_to_remove = []
    for stem_id, stem_data in metadata["stems"].items():
        inst_class = stem_data.get("inst_class", "")
        if inst_class in {"Chromatic Percussion", "Organ", "Synth Lead", "Synth Pad", "Sound Effects", "Ethnic", "Percussive", "Sound effects"}:
            stems_to_remove.append(stem_id)
    
    print(f"Stems to remove: {stems_to_remove}")  # Debugging print

    for stem_id in stems_to_remove:
        stem_file = os.path.join(stems_path, f"{stem_id}.flac")
        if os.path.exists(stem_file):
            print(f"Removing {stem_file}")  # Debugging print
            os.remove(stem_file)
        del metadata["stems"][stem_id]  
    
    with open(metadata_path, "w") as f:
        yaml.dump(metadata, f, default_flow_style=False)
    
    remaining_stems = [os.path.join(stems_path, f"{stem_id}.flac") for stem_id in metadata["stems"] if os.path.exists(os.path.join(stems_path, f"{stem_id}.flac"))]
    print(f"Remaining stems: {remaining_stems}")  # Debugging print
    
    if remaining_stems:
        mixed_audio = None
        for stem_file in remaining_stems:
            print(f"Mixing {stem_file}")  # Debugging print
            audio = AudioSegment.from_file(stem_file)
            mixed_audio = audio if mixed_audio is None else mixed_audio.overlay(audio)
        print("Exporting mixed file")  # Debugging print
        mixed_audio.export(mix_path, format="flac")
    else:
        print(f"Deleting track folder: {track_path}")  # Debugging print
        for root, dirs, files in os.walk(track_path, topdown=False):
            for file in files:
                file_path = os.path.join(root, file)
                print(f"Deleting file: {file_path}")  # Debugging print
                os.remove(file_path)
            for dir in dirs:
                dir_path = os.path.join(root, dir)
                print(f"Deleting dir: {dir_path}")  # Debugging print
                os.rmdir(dir_path)
        print(f"Removing empty track folder: {track_path}")  # Debugging print
        os.rmdir(track_path)

    print(f"Finished processing track: {track}")  # Debugging print
    time.sleep(0.1)  # Prevent excessive CPU usage


Processing track: Track01501
Stems to remove: []
Remaining stems: ['slakh2100_flac_redux/reduced\\Track01501\\stems\\S00.flac', 'slakh2100_flac_redux/reduced\\Track01501\\stems\\S01.flac', 'slakh2100_flac_redux/reduced\\Track01501\\stems\\S03.flac', 'slakh2100_flac_redux/reduced\\Track01501\\stems\\S04.flac', 'slakh2100_flac_redux/reduced\\Track01501\\stems\\S05.flac', 'slakh2100_flac_redux/reduced\\Track01501\\stems\\S06.flac', 'slakh2100_flac_redux/reduced\\Track01501\\stems\\S07.flac', 'slakh2100_flac_redux/reduced\\Track01501\\stems\\S08.flac', 'slakh2100_flac_redux/reduced\\Track01501\\stems\\S10.flac']
Mixing slakh2100_flac_redux/reduced\Track01501\stems\S00.flac
Mixing slakh2100_flac_redux/reduced\Track01501\stems\S01.flac
Mixing slakh2100_flac_redux/reduced\Track01501\stems\S03.flac
Mixing slakh2100_flac_redux/reduced\Track01501\stems\S04.flac
Mixing slakh2100_flac_redux/reduced\Track01501\stems\S05.flac
Mixing slakh2100_flac_redux/reduced\Track01501\stems\S06.flac
Mixing slakh

KeyboardInterrupt: 